* code by Sihyun You (2021.12.28.)
* edit by Jehyun Lee (2021.12.30.)
* revised for mrnIF by Jehyun Lee (2022.01.08.)
* using applicants submitted Eng. name (2022.06.18.)

# Environment settings

* import libraries

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

import requests, json
from docx import Document
from copy import deepcopy

* settings

In [2]:
pd.set_option("mode.chained_assignment", None)
pd.set_option("display.max_columns", None)

# Ready for retrieving

* previous paper analysis

In [3]:
data_paper = "applicants_fill_from0_to1621_ss.xlsx"

df_applicants = pd.read_excel(data_paper, header=0, dtype={"UT":str})
display(df_applicants.head(3))
df_applicants.shape

,전체순번,수험번호 내 순번,수험번호,성명,영문명,논문 내 영문명,논문명,게재일자,DOI,저널구분,저자구분,저널명,출판사,ISSN,저자순위,게재여부,논문구분\n(SCIE),Publication Date,#citation,Publication Year journal impact factor,2020\njournal\nimpact\nfactor,2020 journal impact factor percentile,1st Author,1ST AUTHOR\n(Y/N),Reprint Author,REPRINT AUTHOR\n(Y/N),Source\n(Journal),volume,issue,Notes
0,1,1,0100-000001,고윤지,"Ko, Younji",Younji Ko,Layer-by-Layer Assembly-Based Electrocatalytic...,2021.08.26,10.1002/adfm.202102530,국외SCIE,주저자,"Advanced Functional Materials 31, 2102530, 2021",Wiley-VCH,1616-301X,(1/11),게재,Y,AUG 26 2021,0.0,IF 미발급: 2021년 출판,18.808,95.6586826,"Ko, Younji",Y,NaN,NaN,ADVANCED FUNCTIONAL MATERIALS,31,35,NaN
1,2,2,0100-000001,고윤지,"Ko, Younji",Younji Ko,Hydrophobic and Hydrophilic Nanosheet Catalyst...,2018.04.01,10.1016/j.apsusc.2017.12.037,국외SCIE,주저자,"Applied Surface Science 436, 791, 2018",Elsevier,0169-4332,(1/4),게재,Y,APR 1 2018,3.0,5.155,6.707,97.6190476,"Ko, Younji",Y,NaN,NaN,APPLIED SURFACE SCIENCE,436,NaN,NaN
2,3,3,0100-000001,고윤지,"Ko, Younji",Younji Ko,Carbonization/Interfacial Assembly-Driven Elec...,2022.05.10,Under Revision,국외SCIE,주저자,Energy & Environmental Science,Royal Society of Chemistry,-,(2/5),제출,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Scopus에서 doi 및 title 검색 불가: SemanticScholar 활용 시도


(1621, 30)

In [16]:
# 수험번호
no_appls = df_applicants.sort_values(["수험번호", "성명", "영문명"])["수험번호"].unique()

In [24]:
# h-index 계산
h_indexs = []
name_appls = []
name_appls_en = []
y5_pubs = []
y5_citations = []

for no_appl in no_appls:
    name_appl = df_applicants.loc[df_applicants["수험번호"]==no_appl]["성명"].unique()[0]
    name_appls.append(name_appl)
    
    name_appl_en = df_applicants.loc[df_applicants["수험번호"]==no_appl]["영문명"].unique()[0]
    name_appls_en.append(name_appl_en)
    
    df_appl = df_applicants.loc[df_applicants["수험번호"]==no_appl].sort_values('#citation', ascending=False)
    citations = df_appl['#citation']
    paper_list = np.linspace(1, df_appl.shape[0], df_appl.shape[0], dtype=int)
    h_index = np.array(citations >= paper_list).sum()
    
    h_indexs.append(h_index)
    
    y5_pubs.append(df_appl.shape[0])
    y5_citations.append(int(citations.sum()))

In [25]:
df_h5 = pd.DataFrame({"수험번호":no_appls, "이름":name_appls, "영문명": name_appls_en, "h-index":h_indexs, "총 출판물수": y5_pubs, "총 피인용수": y5_citations})
df_h5

,수험번호,이름,영문명,h-index,총 출판물수,총 피인용수
0,0100-000001,고윤지,"Ko, Younji",3,6,53
1,0100-000003,박세원,"Park, Se-Won",5,10,58
2,0100-000005,이정훈,"Lee, Junghun",3,4,47
3,0100-000007,임석현,"Lim, Suk Hyun",4,9,76
4,0100-000011,전성권,"Jeon, Sungkwon",5,7,101
...,...,...,...,...,...,...
184,0100-000315,박상진,"Park, Sangjin",3,10,26
185,0100-000323,함가현,"Ham, Kahyun",6,10,66
186,0100-000329,임원광,"Lim, Won-Gwang",12,18,692
187,0100-000330,이현우,"Lee, HyunWoo",0,1,0


In [26]:
df_h5.to_excel("applicants_personal.xlsx", index=False)